In [1]:
import glob
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from datetime import datetime, timedelta
from os.path import exists, join

import match_ids


In [2]:
tor_path = "/glade/p/cisl/aiml/jtti_tornado/gridrad_storm_tracks/"
save_path = "/glade/scratch/ggantos/tornado_jtti"


In [3]:
all_dates = match_ids.get_all_pickle_dates(tor_path)
print(f"First ten dates in data: {all_dates[:10]}")


Years in data: ['2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']
First ten dates in data: ['20110108', '20110109', '20110125', '20110224', '20110225', '20110305', '20110318', '20110319', '20110329', '20110330']


In [12]:
cols_drop = ['grid_point_latitudes_deg', 'grid_point_longitudes_deg', 'grid_point_rows', 'grid_point_columns', 'polygon_object_rowcol']

for date in all_dates:
    storm_poly = match_ids.load_track_day(tor_path, date)
    if storm_poly.size == 0:
        print(f"Dropping date: {date}")
        continue
    storm_poly = storm_poly.drop(cols_drop, axis=1)
    storm_poly = gpd.GeoDataFrame(storm_poly, geometry='polygon_object_latlng_deg')
    storm_poly.to_file(join(save_path, f"storm_poly_{date}.geojson"), driver="GeoJSON")


Number of files for 20110108: 120
Number of files for 20110109: 1
Number of files for 20110125: 133
Number of files for 20110224: 145
Number of files for 20110225: 73
Dropping date: 20110225
Number of files for 20110305: 109
Number of files for 20110318: 109
Dropping date: 20110318
Number of files for 20110319: 145
Number of files for 20110329: 181
Number of files for 20110330: 48
Number of files for 20110331: 73
Number of files for 20110409: 109
Number of files for 20110410: 97
Number of files for 20110414: 204
Number of files for 20110415: 288
Number of files for 20110416: 181
Number of files for 20110421: 121
Number of files for 20110424: 145
Number of files for 20110426: 180
Number of files for 20110427: 157
Number of files for 20110501: 73
Number of files for 20110522: 97
Number of files for 20110524: 145
Number of files for 20110526: 121
Number of files for 20110528: 109
Number of files for 20110529: 97
Number of files for 20110531: 12
Number of files for 20110601: 193
Number of 

In [ ]:
# def lon_to_web_mercator(lon):
#     k = 6378137
#     return lon * (k * np.pi / 180.0)

# def lat_to_web_mercator(lat):
#     k = 6378137
#     return np.log(np.tan(((90 + lat) * np.pi / 360.0))) * k


In [ ]:
# def convert_polygon_to_boundary(df):
#     xs = []
#     lons = []
#     ys = []
#     lats = []
#     for i in df.index:
#         x, y = df.loc[i, 'polygon_object_latlng_deg'].exterior.coords.xy
#         xs.append(lon_to_web_mercator(np.array(x) - 360))
#         lons.append(np.array(x) - 360)
#         ys.append(lat_to_web_mercator(np.array(y)))
#         lats.append(np.array(y))
#     df['x'] = xs
#     df['lons'] = lons
#     df['y'] = ys
#     df['lats'] = lats
#     return df

In [7]:
for date in all_dates[:10]:
    storm_ds = match_ids.load_patch_day(tor_path, date)
    if storm_ds is None:
        print(f"Dropping date: {date}")
        continue
    storm_poly = match_ids.load_track_day(tor_path, date)
#     storm_poly['valid_time'] = pd.to_datetime(storm_poly['valid_time_unix_sec'], unit='s')
#     storm_poly = convert_polygon_to_boundary(storm_poly)
#     storm_poly = storm_poly.drop(cols_drop, axis=1)
    poly_id = storm_poly["full_id_string"] + "_" + storm_poly["valid_time_unix_sec"].astype(str)
    ds_id = pd.Series(storm_ds["full_storm_id_strings"].values).str.decode("utf-8") + "_" + pd.Series(storm_ds["storm_times_unix_sec"].values).astype(str)
    overlap = np.intersect1d(poly_id, ds_id)
    print(date, poly_id.size, ds_id.size, overlap.size)
    

Number of files for 20110108: 120
20110108 421 420 420
Number of files for 20110109: 1
20110109 1 1 1
Number of files for 20110125: 133
20110125 595 573 573
Number of files for 20110224: 145
20110224 3083 3073 3073
Dropping date: 20110225
Dropping date: 20110305
Dropping date: 20110318
Number of files for 20110319: 145
20110319 724 722 722
Number of files for 20110329: 181
20110329 5562 5552 5552
Number of files for 20110330: 48
20110330 88 88 88
